## 카카오API를 통한 부산 충전소 위치 위도,경도 추출

### 부산시내 이용제한이 없는 충전소 위치 위도,경도 추출

In [4]:
import pandas as pd
import re
import requests, json

# 위경도 찾을 파일 불러오기
file_path = './data/newdata/3. 주유소/부산_주유소_주소.csv'
busan_oil = pd.read_csv(file_path,encoding='euc-kr')

# 주소 전처리
pattern = r'\([^)]*\)'
text = []

for i in range(0, len(busan_oil['주소']),1):
    text.append(re.sub(pattern=pattern, repl='', string= busan_oil['주소'][i]))

text
busan_oil['주소'] = text

# 카카오API를 사용하여 주소 -> 좌표 변환
def get_location(address):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
  # 'KaKaoAK '는 그대로 두시고 개인키만 지우고 입력해 주세요.
  # ex) KakaoAK 6af8d4826f0e56c54bc794fa8a294
  headers = {"Authorization": "KakaoAK 57da1f1bcfa0f4b06f0350e852287856"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  address = api_json['documents'][0]['address']
  crd = {"위도": str(address['y']), "경도": str(address['x'])}
  address_name = address['address_name']

  return crd

# 파일에서 주소를 위경도로 변환해서 crd에 넣기
crd = []

for i in range(0, len(busan_oil['주소']), 1):
    try:
        crd.append(get_location(busan_oil['주소'][i]))
    except:
        crd.append({'위도':0,'경도':0})

# crd를 데이터프레임화 하여 원래 파일에 붙이기
busan_oil_final = pd.DataFrame(crd)
busan_oil_final = pd.concat([busan_oil,busan_oil_final], axis=1)
busan_oil_final

,주소,위도,경도
0,부산광역시 강서구 낙동북로 310,35.2125066561207,128.962571748259
1,부산광역시 강서구 호계로 195,35.2088470489914,128.890148280028
2,부산광역시 강서구 경전철로188번길 95,35.1986693564441,128.966594429738
3,부산광역시 강서구 르노삼성대로 61,35.0954080178918,128.883215288804
4,부산광역시 강서구 공항로 459,35.1490640544657,128.951890961126
...,...,...,...
386,부산광역시 연제구 연수로 84,35.1752946402192,129.079865906103
387,부산광역시 연제구 과정로 120,35.1822674356657,129.107462250933
388,부산광역시 연제구 반송로 68,35.1898011709,129.086950032177
389,부산광역시 연제구 중앙대로 1179,35.192802640629,129.079556285714


In [8]:
# 위도,경도 출력되지 않은 주소 필터링
busan_oil_final_NaN = busan_oil_final[busan_oil_final['위도'] == 0]
busan_oil_final_NaN

,주소,위도,경도
9,부산광역시 강서구 송정동 산71-2,0,0
51,일광면 삼성리 25,0,0
62,장안읍 장안리 산78-1,0,0
119,부산광역시 남구 유엔로 235,0,0
200,부산광역시 영도구 태종로 782,0,0
216,부산광역시 해운대구 수영강변대로 462,0,0


In [12]:
# 데이터 정제를 위한 내보내기
busan_oil_final.to_excel('./data/newdata/3. 주유소/부산_주유소_위경도_추출_결측치.xlsx')